In [2]:
import tmilib
reload(tmilib)
from tmilib import *

In [8]:
def get_focused_tab(data):
  windows = data['windows']
  for window in windows:
    focused = window['focused']
    if not focused:
      continue
    tabs = window['tabs']
    for tab in tabs:
      if not tab['highlighted']:
        continue
      if not tab['selected']:
        continue
      return tab['url']

for filename in list_logfiles()[:1]:
  size_bytes = os.stat(filename).st_size
  size_megabytes = float(size_bytes) / (1000.0*1000.0)
  if size_megabytes > 0.1:
    continue
  print filename
  output = []
  prev_active_site = ''
  for data in iterate_data(filename):
    #evt_types.add(data['evt'])
    #print data
    current_active_site = get_focused_tab(data)
    # things to handle
    # idle_changed (if idlestate == idle or locked), https://developer.chrome.com/extensions/idle#type-IdleState
    # default detection interval is 60 seconds for idle
    # still_browsing after 30 seconds, or lack thereof
    #if current_active_site == None: # no windows focused
    #  
    if get_focused_tab(data) == None:
      print data
      break
    #break

/home/gkovacs/tmi-data/local_2016-03-23_19:13:45-07:00/logs_Eq7EExfolE.json
{u'ver': u'1', u'windows': [{u'tabs': [{u'status': u'complete', u'index': 0, u'title': u'Extensions', u'pinned': False, u'selected': False, u'audible': False, u'highlighted': False, u'mutedInfo': {u'muted': False}, u'width': 1366, u'url': u'chrome://extensions/', u'windowId': 1, u'active': False, u'favIconUrl': u'chrome://theme/IDR_EXTENSIONS_FAVICON@2x', u'height': 667, u'id': 2, u'incognito': False}, {u'status': u'complete', u'index': 1, u'title': u'https://tmi.stanford.edu/mturk3.html?username=A21NRMZFK7QCJW', u'pinned': False, u'selected': True, u'audible': False, u'highlighted': True, u'mutedInfo': {u'muted': False}, u'width': 1366, u'url': u'https://tmi.stanford.edu/mturk3.html?username=A21NRMZFK7QCJW', u'windowId': 1, u'active': True, u'height': 667, u'id': 4, u'incognito': False}, {u'status': u'complete', u'index': 2, u'openerTabId': 4, u'title': u'BrowserLog - Chrome Web Store', u'pinned': False, u'sel

In [4]:
evt_types = set()
for filename in list_logfiles():
  size_bytes = os.stat(filename).st_size
  size_megabytes = float(size_bytes) / (1000.0*1000.0)
  if size_megabytes > 0.1:
    continue
  print filename
  for data in iterate_data_compressed(filename):
    evt_types.add(data['evt'])
  #break
  #size_bytes = os.stat(x).st_size
  #size_megabytes = float(size_bytes) / (1000.0*1000.0)
  #print size_megabytes

/home/gkovacs/tmi-data/local_2016-03-23_19:13:45-07:00/logs_Eq7EExfolE.json
/home/gkovacs/tmi-data/local_2016-03-23_19:13:45-07:00/logs_3H5aGCb1Km.json
/home/gkovacs/tmi-data/local_2016-03-23_19:13:45-07:00/logs_vbZrd5LD5J.json
/home/gkovacs/tmi-data/local_2016-03-23_19:13:45-07:00/logs_pQiVSc5Ta5.json
/home/gkovacs/tmi-data/local_2016-03-23_19:13:45-07:00/logs_gtg8G9PJk7.json
/home/gkovacs/tmi-data/local_2016-03-23_19:13:45-07:00/logs_r9pox860Kx.json
/home/gkovacs/tmi-data/local_2016-03-23_19:13:45-07:00/logs_ZIpD1khEqv.json
/home/gkovacs/tmi-data/local_2016-03-23_19:13:45-07:00/logs_kEVg7v0LcS.json
/home/gkovacs/tmi-data/local_2016-03-23_19:13:45-07:00/logs_cTggWOIxwe.json
/home/gkovacs/tmi-data/local_2016-03-23_19:13:45-07:00/logs_epgLZz5GYr.json
/home/gkovacs/tmi-data/local_2016-03-23_19:13:45-07:00/logs_NlUyZzFe88.json
/home/gkovacs/tmi-data/local_2016-03-23_19:13:45-07:00/logs_uVep19IpQg.json
/home/gkovacs/tmi-data/local_2016-03-23_19:13:45-07:00/logs_JgGfKNPVdz.json
/home/gkovac

In [15]:
print evt_types

set([u'tab_moved', u'tab_replaced', u'tab_zoomchange', u'browser_focus_changed', u'tab_activated', u'tab_created', u'idle_changed', u'tab_updated', u'window_focus_changed', u'tab_attached', u'window_created', u'tab_highlighted', u'window_closed', u'tab_removed', u'still_browsing', u'tab_detached'])
